In [1]:
#!pip install ocs-academic-hub
#!pip uninstall -y ocs_academic_hub
#!pip list

In [2]:
#!git clone https://github.com/cfoisy-osisoft/academic-hub.git

In [3]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e12

In [4]:
from ocs_academic_hub import OMFClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse

C:\Users\zcheng\Anaconda3\lib\site-packages\typeguard\__init__.py:906: UserWarning: no type annotations present -- not typechecking ocs_academic_hub.omf_client.OMFClient.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


# Data frame generation

In [5]:
start = pd.to_datetime('2021-03-29 15:32:00')
end = pd.to_datetime('2021-03-29 15:35:00')
freq = 200 #Hz
column = 4

In [6]:
time_range = pd.date_range(start, end, freq=f'{1/freq*1000000000}ns')

np.random.seed(seed=1111)
data = np.random.uniform(1, high=100, size=(len(time_range), column))

df = pd.DataFrame({'Timestamp': time_range}) 
#df = df.set_index('Timestamp')
i = 0
while i < column:
    df[f'col{i + 1}'] = data[:,i]
    i += 1
print(df)

                    Timestamp       col1       col2       col3       col4
0     2021-03-29 15:32:00.000  10.459371  92.575366  35.013769  31.737217
1     2021-03-29 15:32:00.005   1.198974  24.323878  24.541380  73.855671
2     2021-03-29 15:32:00.010  50.051340  78.658110  13.524125  61.058283
3     2021-03-29 15:32:00.015  47.145977  24.476080  44.080759  25.123480
4     2021-03-29 15:32:00.020  39.000151  84.000975  65.863288  15.696221
...                       ...        ...        ...        ...        ...
35996 2021-03-29 15:34:59.980  70.735944   9.417489  42.654820  33.593369
35997 2021-03-29 15:34:59.985  47.261051  39.804569   4.691471   6.108635
35998 2021-03-29 15:34:59.990  32.342663  17.350372  95.945134  80.430994
35999 2021-03-29 15:34:59.995  90.784290  26.558765  42.095978  45.002198
36000 2021-03-29 15:35:00.000  67.377418  60.999185  34.538138  19.687933

[36001 rows x 5 columns]


# Schedular 

In [7]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

# Send OMF Message to EDS

In [8]:
from time import sleep

asset = 'test'
omf_client = OMFClient()

def event():
    df_rt=df[abs(df['Timestamp'] - pd.Timestamp.now())<=timedelta(seconds=1/freq)]
    omf_client.update_tags(df_rt, asset)
    #print(df_rt)
    
if (pd.Timestamp.now()>start):
    print ("starting...")
    rt = RepeatedTimer(1/freq, event) # it auto-starts, no need of rt.start()
    try:
        sleep((end-pd.Timestamp.now()).total_seconds()) # your long-running job goes here...
    finally:
        rt.stop() # better in a try/finally block to make sure the program ends!
        print ("stopped!!!")

starting...
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:10.040000 to 2021-03-29 15:32:10.045000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:10.060000 to 2021-03-29 15:32:10.065000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:10.075000 to 2021-03-29 15:32:10.080000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:10.090000 to 2021-03-29 15:32:10.095000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:10.105000 to 2021-03-29 15:32:10.110000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:10.120000 to 2021-03-29 15:32:10.125000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:10.135000 to 2021-03-29 15:32:10.140000
>

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:11.165000 to 2021-03-29 15:32:11.170000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:11.200000 to 2021-03-29 15:32:11.205000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:11.230000 to 2021-03-29 15:32:11.235000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:11.245000 to 2021-03-29 15:32:11.250000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:11.265000 to 2021-03-29 15:32:11.270000
>> [column definitions OK]
>> processing row: [last rows 8]
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:11.295000 to 2021-03-29 15:32:11.300000
>> [column definitions OK]
>> processing row: [last rows 8]
>> new tag(s): ['x.test.col1', 'x.

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:12.500000 to 2021-03-29 15:32:12.505000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:12.530000 to 2021-03-29 15:32:12.535000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:12.545000 to 2021-03-29 15:32:12.550000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:12.560000 to 2021-03-29 15:32:12.565000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:12.575000 to 2021-03-29 15:32:12.580000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:12.590000 to 2021-03-29 15:32:12.595000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:12.605000 to 2021-03-29 15:32:12.610000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:14.025000 to 2021-03-29 15:32:14.030000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:14.045000 to 2021-03-29 15:32:14.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:14.080000 to 2021-03-29 15:32:14.085000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:14.095000 to 2021-03-29 15:32:14.100000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:14.110000 to 2021-03-29 15:32:14.115000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:14.125000 to 2021-03-29 15:32:14.130000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:14.140000 to 2021-03-29 15:32:14.145000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:15.280000 to 2021-03-29 15:32:15.285000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:15.295000 to 2021-03-29 15:32:15.300000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:15.310000 to 2021-03-29 15:32:15.315000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:15.325000 to 2021-03-29 15:32:15.330000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:15.350000 to 2021-03-29 15:32:15.355000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:15.370000 to 2021-03-29 15:32:15.375000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-03-29 15:32:15.385000 to 2021-03-29 15:32:15.390000
>> new tag(s)

...


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub do

[last rows 8]
>> processing row: >> processing row: [last rows 8][last rows 8]>> processing row: 
[last rows 8]
[last rows 8]
[last rows 8]

[last rows 8][last rows 8][last rows 8]
[last rows 8][last rows 8]
[last rows 8]
>> processing row: [last rows 8][last rows 8]
[last rows 8]
[last rows 8][last rows 8]


[last rows 8]


[last rows 8][last rows 8]


[last rows 8]


[last rows 8][last rows 8]





>> [column definitions OK]>> [column definitions OK]
>> processing row: [last rows 8]
>> [column definitions OK]>> [column definitions OK]>> [column definitions OK]
>> processing row: [last rows 8]>> [column definitions OK]
>> processing row: [last rows 8]
>> [column definitions OK]>> [column definitions OK]>> [column definitions OK]
>> processing row: [last rows 8]>> [column definitions OK]
>> [column definitions OK]>> [column definitions OK]>> [column definitions OK]>> [column definitions OK]
>> [column definitions OK]
>> processing row: [last rows 8]>> processing row: >> [column definit

[last rows 8][last rows 8]


Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

[last rows 8]


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

[last rows 8]>> processing row: 
[last rows 8]
[last rows 8]




[last rows 8]



>> processing row: [last rows 8][last rows 8][last rows 8]




Loading-Extract-Transfer to Hub done, status OK, #rows = 2

>> processing row: 
[last rows 8]

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

[last rows 8]
[last rows 8]
[last rows 8]


Loading-Extract-Transfer to Hub done, status OK, #rows = 2






Loading-Extract-Transfer to Hub done, status OK, #rows = 2

L


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2




Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer t


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer to Hub done, status OK, #rows = 2



Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2


Loading-Extract-Transfer to Hub done, status OK, #rows = 2

Loading-Extract-Transfer